## **ORM**

In [ ]:
# RDBMS의 단점
# 원래 데이터의 흐름보다는 테이블에 더 관점을 주고, 그로 인해서 데이터를 조작하는데 있어서 수많은 sql 문들이 필요함을 봤음.

In [ ]:
# ORM 이란?
# Object Relational Mapping
# programming 테크닉. 즉, DB랑 상관없음.
# 그런데 데이터베이스와 비즈니스 로직이 서로 다르다는 점에서 필요성이 드러남.
# ORM은 OOP(객체지향)관점에서 데이터들을 오브젝트로 매핑해줌.
# 엔티티간의 관계를 오브젝트화 해서 데이터 관점에서 바라보게 해준다

In [ ]:
# ERD -> 엔티티, 관계를 나타낸 다이어그램
# 이것만 보고는 테이블이 어떻게 이루어져 있는지는 모름.
# 2개, 3개(관계까지 테이블로 만들어서) 테이블이 될 수 있음. 방법은 여러가지. ERD만 보고 테이블이 어떻게 구성될 지 판단하기 어려움.
# 데이터 관점에서 봤을 떄 고객과 은행계좌는 다른 엔티티.

In [ ]:
# ORM 은 접근하는 방식이 다름. 객체지향 프로그램 처럼 프로퍼티 사용 등 특징이 있음.
# DB에 구현해놓은 것과 우리가 이용하는 방식이 다르기 떄문에, 코드도 복잡해지고 시간도 많이 걸리게 되는데,
# 그것을 줄여보자는 취지에서 ORM을 사용하게 됨.
# 즉 미스매치. 실제 RDB에 저장되어 있는 것들과 우리가 사용하는 것이 미스매치.
# ORM이란 기능/테크닉은 데이터베이스와 비즈니스 오브젝트 사이에 매핑을 해줌.
# 즉 이제 우리는 관계형 DB를 할 줄 알기 때문에 ORM 을 배울 것.

In [ ]:

#! ORM을 쓰는 근본적인 이유
# 프로그래머들이 데이터베이스를 어떻게 관리하고 이용하는지 보다는 로직 별로 프로그래밍 하는데 더 집중할 수 있도록 필요함
# 단점으로는 overhead가 있음.
# 내가 object를 건들면 ORM이 query로 처리해줌. 그걸 또 처리하는 한단계가 더 있어서 overhead가 존재함.

In [ ]:
# 모델과 실제 구현 결과물은 얼마든지 다를 수 있다!!!
# 사용자 태그 예시(인스타그램과 유사)

In [ ]:
!ls

In [ ]:
1. 새 게시물 생성
2. 생성 시, 생성자는 사용자태그를 0 ~ N개 만들 수 있음
    -3. 사용자태그가 해시태그 풀에 있는지 확인
    -4. 확인이 됐으면, 새 게시물과 해시태그 관계 만들어주기
    -5. 해시태그 풀에 있는 빈도정보 +1
[여기까지 게시물 하나 작성하는 상황]

In [ ]:
<<정보>>

posting - pk, title, content, date

hashtag - pk, name, count

posting - hashtag : posting.pk, hashtag.pk

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect('sns2.db')
cur = con.cursor()

In [ ]:
cur.execute('SELECT CURRENT_TIMESTAMP')
cur.fetchall()

In [ ]:
cur.executescript('''
    DROP TABLE IF EXISTS posting;
    CREATE TABLE posting(
        pk INTEGER PRIMARY KEY,
        title TEXT,
        content TEXT,
        regdate DATE DEFAULT CURRENT_TIMESTAMP
    );
    DROP TABLE IF EXISTS hashtag;
    CREATE TABLE hashtag(
        pk INTEGER PRIMARY KEY,
        name TEXT,
        count INTEGER DEFAULT 0
    );
    DROP TABLE IF EXISTS poshas;
    CREATE TABLE poshas(
        fk1 INTEGER NOT NULL,
        fk2 INTEGER NOT NULL
    );
''')

In [ ]:
# def addPosting(title, content, *hashtag) -> ORM을 위해 나중에(commit으로 잘 돌아가는지 확인 후) 감싸줄 것.

# 1단계 - 새 게시물 생성
cur.execute('INSERT INTO posting(title, content) VALUES(?, ?)',
            ['제목1', '내용1'])
pid = cur.lastrowid
pid

In [ ]:
cur.execute('SELECT * FROM posting')
cur.fetchall()

In [ ]:
hashtag = ['태그1', '태그2']
tagids = list()

# 2단계 - 사용자 태그 만들기
#   3단계 - 사용자 태그가 해시태그 풀에 있는지 확인
for tag in hashtag:
    cur.execute('SELECT pk FROM hashtag WHERE name=?', [tag])
    # print(cur.fetchone())   --> 없으면 None
    tid = cur.fetchone()
    if tid is not None:
        tagids.append(tid[0])

tagids

In [ ]:
tid

In [ ]:
#   4단계 - 확인이 됐으면, 새 게시물과 해시태그 관계 만들어주기
#   5단계 - 해시태그 풀에 있는 빈도정보 +1

for tid in tagids:  
    cur.execute('''
        INSERT INTO poshas VALUES(?,?)
    ''', [pid, tid])
    
    cur.execute('''
        UPDATE hashtag
        SET count = count + 1
        WHERE pk = ?                
    ''', [tid])

In [ ]:
# 태그풀 생성
tags = [['태그1'], ['태그2'], ['태그3']]
cur.executemany('INSERT INTO hashtag(name) VALUES(?)', tags)

In [ ]:
con.commit()
#! 여기까지 잘 돌아감을 확인(실제 내 DB에 반영이 되면)하면, 함수로 감싸주면 됨
# 꼭 Query문을 다 작성해서 확인을 했으면, 물리적으로 DB에 반영해주어야 한다. 그것이 COMMIT

In [ ]:
# 여기까지 생성. 이제는 수정

In [ ]:
cur.execute('SELECT * FROM poshas')
cur.fetchall()

In [ ]:
cur.execute('SELECT * FROM posting')
cur.fetchall()

In [ ]:
cur.execute('SELECT * FROM hashtag')
cur.fetchall()

In [ ]:
cur.execute('''
    SELECT posting.title, posting.content, posting.regdate, hashtag.name FROM poshas
    INNER JOIN posting ON posting.pk = fk1
    INNER JOIN hashtag ON hashtag.pk = fk2
''')
cur.fetchall()

In [ ]:
pk

In [ ]:
pid

In [ ]:
# 태그 정보만 필요하다면,
cur.execute('''
    SELECT hashtag.name
    FROM poshas
    INNER JOIN posting ON posting.pk = fk1
    INNER JOIN hashtag ON hashtag.pk = fk2
    WHERE fk1 = ?
''', [pid])
cur.fetchall()
# 1번 포스팅에 해당되는 태그들만 가져올 수 있음. 위에서 WHERE 절만 추가한 것.

In [ ]:
# 제목1, 내용1, 태그1, 태그2
# -> 제목1-1, 내용1, 태그1, 태그3
# 이렇게 바꿔보자!

In [ ]:
cur.execute('UPDATE posting SET title=?, content=?',
            ['제목1-1', '내용1'])

In [ ]:
cur.rowcount

In [ ]:
hashtag = ['태그1', '태그3']
oldtag = list()
tagids = list()

cur.execute('SELECT fk2 FROM poshas WHERE fk1=?', [pid])
for row in cur.fetchall():
    oldtag.append(row[0])
    # 기존의 태그 목록
    
for tag in hashtag:
    cur.execute('SELECT pk FROM hashtag WHERE name=?', [tag])
    # print(cur.fetchone())   --> 없으면 None
    tid = cur.fetchone()
    if tid is not None:
        tagids.append(tid[0])

oldtag, tagids

In [ ]:
for tid in oldtag:
    if tid not in tagids:  # 없으면
        cur.execute('DELETE FROM poshas WHERE fk1=? AND fk2=?',
                    [pid, tid])
        
        cur.execute('UPDATE hashtag SET count = count - 1 WHERE pk=?',
                    [tid])
        
        

In [ ]:
for tid in tagids:
    if tid not in oldtag:  # 없으면
        cur.execute('INSERT INTO poshas VALUES(?,?)',
                    [pid, tid])
        
        cur.execute('UPDATE hashtag SET count = count + 1 WHERE pk=?',
                    [tid])
        
        

In [ ]:
cur.execute('SELECT * FROM poshas')
cur.fetchall()

In [ ]:
cur.execute('SELECT * FROM hashtag')
cur.fetchall()

In [ ]:
con.close()

#### **뷰**

In [ ]:
CREATE VIEW 뷰 이름 AS
SQL문 -> JOIN
프랜차이즈 -> 지점별 매출현황 -> JOIN(CITY, SUPPLIER, SELLS, PART)
# 이런 것들을 계속 조회할 필요가 있을 때 뷰로 만들어 놓으면 테이블 select 하듯이 하면 실시간으로 검색 결과가 나옴.
# 너무 많은 뷰를 만들지만 않으면 상관 없음.

In [ ]:
# 지금까지는 우리가 SQL문을 알아야 했지만, 이제는 그럴 필요가 없음. 파이썬 코드를 통해 접근할 것.
# 인스턴스를 생성하고 값을 집어넣으면 DB에 적용되도록.
# 각각의 인스턴스는 테이블의 개별 로우를 의미. 데이터베이스에서 데이터를 꺼낼 때 로우 단위로 꺼내기 떄문.
# 우리는 SQLAlchemy 를 사용할 것임.
# ORM을 이용하면 동일한 코드로 여러 DB를 사용 가능함.

## **SQLAlchemy**

In [ ]:
# 파이썬으로 만들어짐.
# SQLAlchemy의 구조 -> DBAPI(각 DB 종류마다에 붙는 API) -> Dialect는 그 DBAPI와 통신 
# -> 대화형으로 주고받기 위해 connection pooling -> 그 위의 engine은 실제 오브젝트를 통해서 코드레벨로 접근
# 했을 때 객체에 조작이 일어난 행위를 sql문으로 해석을 함 -> 객체와 sql문을 연결지어주는 애들이 바로
# '스키마/타입' 과 'SQL Expression Language' -> 이제부터 인스턴스를 받아서 객체 단위(하나의 로우를 나타냄)로
# 접근하면 됨. 이 전체가 Core 단. 이 core 단을 감싸는 mapper 가 ORM. 이를 이해하기 위해서 밑에 있는 core 부분을 다뤄보자
# 우리가 ORM을 사용하지 않으면 DB를 바꿀 때마다 여러가지를 바꿔야함.

In [ ]:
# 자료 참고. -> 꼭 읽어보기
# 항상 engine 부터 시작함. 커서, 커넥션 등 총괄함
# engine 이 starting point

In [ ]:
# Engine - 스타팅 포인트
# Dialect - 일반적인 SQL문과 DB 커맨드를 특정한 DBAPI에 관점에 맞춰 해석한다.
# Connection Pool - 메모리에 DB로부터 가져온 것들을 모아둔 Pool

In [ ]:
# create_engine 형식에 대한 설명!!
# sqlite3는 단일 fileDB -> port(서버 리스이기 때문)/database/사용자 계정(혼자 사용하기 때문) 없음.
# path(file이기 때문)는 필요
# sqlite://<nohostname>/<path> 식으로 쓰면됨
# 'echo = True' -> 에코 옵션은 중간의 히스토리를 보여줌. 어느 순간에 물리적인 DB로 기록되는지 볼 거기 때문에 켜 놓을 것.
# 'lazy connecting' -> 저장 시 접근 일어나는 것 아님. 지금부터 하는 작업은 DB로부터 가져온 데이터로 하는 것 같지만
# 실질적으로는 인스턴스 형태를 띈 객체를 통해 데이터로 접근. 따라서 객체가 insert가 되는 그 시점에 connecting 이 이루어짐.

In [ ]:
# table, column, MetaData(메모리에서 객체로 관리하기 때문에 그 객체가 등록된 곳이 메타데이터. 가장 중요함)
# 메타데이터를 엔진에 바인딩 시킴.
# 메타데이터에 변화가 일어나 정보 갱신을 요구하면 -> 엔진에서 값의 변화를 인지하고 커넥션 풀을 통해 접근하고 
# -> dialect로 해석하여 값을 보냄.

In [ ]:
# 따라서 기존 RDB의 테이블을 알아야 하고, 메타데이터에 등록된 인스턴스들도 알아야 함.
# 싱크가 맞아야 하고 그렇지 않으면 오류가 나게 됨

##### 예제 시작

In [ ]:
# '스키마'에 table, column, metadata가 정의되어 있음
# '타입'에 integer / string 등 정의되어 있음 => 즉 ORM에서 정의된 것
# 먼저 engine 만들어 줘야 함.
# lazy connecting이라 아무 일도 일어나지 않음.

In [ ]:
# ORM에서는 FK를 꼭 설정해줘야 함(여러 RDB를 관리해야 하기 때문에 엄격)

In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'2.0.5.post1'

In [276]:
from sqlalchemy import create_engine
from sqlalchemy.schema import MetaData, Table, Column, ForeignKey
from sqlalchemy.types import Integer, String
# 위에서 설명한 개념을 볼 수 있다..!

In [277]:
# starting point -> 항상 엔진 생성!!
engine = create_engine('sqlite:///orm2.db', echo=True)
# echo=True 때문에 로고가 남을 것.
# 엔진 생성은 아무 작업이 아니라 로고가 안남음

In [3]:
# 엔진을 닫는 명령어
engine.dispose()

In [275]:
# 초기화
con.close()
meta.clear()
engine.dispose()

2023-03-08 00:37:04,499 INFO sqlalchemy.engine.Engine ROLLBACK


In [169]:
!ls
# 위에서 커넥트 한순간 DB가 생성되었어야 하는데 없음. -> lazy connecting

03.02(오후).ipynb           playlist.db
03.03(오전).ipynb           playlist2.db
03.03(오후).ipynb           playlist4.db
03.06(오전).ipynb           project_exercise.py
03.06(오후).ipynb           sns.db
03.07(오전).ipynb           sns2.db
03.07(오후).ipynb           test.db
example1.db                 test1.db
example2.db                 강의자료
example3.db


In [278]:
# meta = MetaData(engine) -> 엔진을 연결한 사례
# MetaData 는 테이블 객체의 집합
# engine 혹은 connection 을 선택적으로 바인딩 할 수 있음.
# 바인딩 되면 SQL문을 실행할 수 있음
meta = MetaData()

In [279]:
meta.tables

FacadeDict({})

In [280]:
Table('T_USER', meta,
      Column('PK', Integer, primary_key=True),
      Column('NAME', String, nullable=False),extend_existing=True)

Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', String(), table=<T_USER>, nullable=False), schema=None)

In [173]:
# 테이블 객체를 하나 생성 -> meta 등록 -> 아직은 실제 물리적 DB 반영 X
!ls # 테이블을 만들었는데 여전히 DB가 없음

03.02(오후).ipynb           playlist.db
03.03(오전).ipynb           playlist2.db
03.03(오후).ipynb           playlist4.db
03.06(오전).ipynb           project_exercise.py
03.06(오후).ipynb           sns.db
03.07(오전).ipynb           sns2.db
03.07(오후).ipynb           test.db
example1.db                 test1.db
example2.db                 강의자료
example3.db


In [282]:
meta.tables
# MetaData에는 들어가 있음

FacadeDict({'T_USER': Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', String(), table=<T_USER>, nullable=False), schema=None)})

In [281]:
meta.bind = engine   # 바인드 시키기

In [283]:
meta.create_all(engine)
# 물리적으로 등록

2023-03-08 00:37:42,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:37:42,451 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2023-03-08 00:37:42,451 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:37:42,452 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_USER")
2023-03-08 00:37:42,453 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:37:42,454 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" VARCHAR NOT NULL, 
	PRIMARY KEY ("PK")
)


2023-03-08 00:37:42,454 INFO sqlalchemy.engine.Engine [no key 0.00034s] ()
2023-03-08 00:37:42,456 INFO sqlalchemy.engine.Engine COMMIT


In [177]:
!ls

03.02(오후).ipynb           orm1.db
03.03(오전).ipynb           playlist.db
03.03(오후).ipynb           playlist2.db
03.06(오전).ipynb           playlist4.db
03.06(오후).ipynb           project_exercise.py
03.07(오전).ipynb           sns.db
03.07(오후).ipynb           sns2.db
example1.db                 test.db
example2.db                 test1.db
example3.db                 강의자료


In [178]:
meta.tables['T_USER'] # 테이블

Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', String(), table=<T_USER>, nullable=False), schema=None)

In [284]:
from sqlalchemy.sql import select, insert, update

In [285]:
print(select(meta.tables['T_USER']))
# 이것은 메모리에서 일어난 작업.

SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER"


In [286]:
print(meta.tables['T_USER'].insert())

INSERT INTO "T_USER" ("PK", "NAME") VALUES (:PK, :NAME)


In [287]:
User = meta.tables['T_USER'] # 객체 만들어주기

In [288]:
print(insert(User), select(User), User.update())
# 사용 방식이 2가지가 있음을 알려줌

INSERT INTO "T_USER" ("PK", "NAME") VALUES (:PK, :NAME) SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER" UPDATE "T_USER" SET "PK"=:PK, "NAME"=:NAME


In [289]:
print(User.insert(), User.select(), update(User))

INSERT INTO "T_USER" ("PK", "NAME") VALUES (:PK, :NAME) SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER" UPDATE "T_USER" SET "PK"=:PK, "NAME"=:NAME


In [15]:
# 여기까지는 DB랑 관련 없음. 메모리에서 객체만 다룬 것 -> 후에 뭔가 일어나야 함

In [290]:
print(meta.tables['T_USER'].insert().values(NAME='아무나'))

INSERT INTO "T_USER" ("NAME") VALUES (:NAME)


In [291]:
# 위와 같은 방식임
print(insert(User).values(NAME='아무나'))

INSERT INTO "T_USER" ("NAME") VALUES (:NAME)


In [292]:
print(User.insert().values(NAME='아무나').compile())

INSERT INTO "T_USER" ("NAME") VALUES (:NAME)


In [293]:
print(User.insert().values(NAME='아무나').compile().params)

{'NAME': '아무나'}


In [294]:
con = engine.connect()
# Connection
# 실제로 DBAPI 에 연결하는 것

In [295]:
cur = con.execute(insert(User).values(NAME='아무나'))
# DB 에 반영하느라 COMMIT 이 뜸
cur.inserted_primary_key

2023-03-08 00:38:26,369 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:38:26,370 INFO sqlalchemy.engine.Engine INSERT INTO "T_USER" ("NAME") VALUES (?)
2023-03-08 00:38:26,371 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ('아무나',)


(1,)

In [296]:
con.execute(User.select()).fetchall()
# 로그가 뜨는 것을 보면 데이터 베이스를 왔다갔다 하는 것

2023-03-08 00:38:35,157 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER"
2023-03-08 00:38:35,158 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ()


[(1, '아무나')]

In [ ]:
# ResultProxy -> 각 DB의 cursor 행위를 함
# fetchone(), fetchall()

In [ ]:
# 로그를 봐야 한다!!!

In [191]:
print(User.c.PK == 1)
# 파이썬 코드로 치는 비교연산 -> 그 결과물은 실제 sql 문에 들어가는 where절을 서술한 부분이 됨.

"T_USER"."PK" = :PK_1


In [192]:
from sqlalchemy.sql import and_, or_, between

In [297]:
print(and_(User.c.PK == 1, User.c.NAME == 1))
# AND 를 쓰고 싶을 때 -> SQL 문에서 이렇게 해석함을 보면 됨.

"T_USER"."PK" = :PK_1 AND "T_USER"."NAME" = :NAME_1


In [ ]:
# 이들을 이용해서 USER 클래스에서 부여받은 객체에 사용할 수 있음.

In [307]:
Table('T_ADDRESS', meta, 
      Column('PK', Integer, primary_key=True),
      Column('ADDR', String),
      Column('FK', Integer, nullable=False),
      extend_existing=True)

Table('T_ADDRESS', MetaData(), Column('PK', Integer(), table=<T_ADDRESS>, primary_key=True, nullable=False), Column('ADDR', String(), table=<T_ADDRESS>), Column('FK', Integer(), table=<T_ADDRESS>, nullable=False), schema=None)

In [299]:
len(meta.tables), meta.tables

(2,
 FacadeDict({'T_USER': Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', String(), table=<T_USER>, nullable=False), schema=None), 'T_ADDRESS': Table('T_ADDRESS', MetaData(), Column('PK', Integer(), table=<T_ADDRESS>, primary_key=True, nullable=False), Column('ADDR', String(), table=<T_ADDRESS>), Column('FK', Integer(), table=<T_ADDRESS>, nullable=False), schema=None)}))

In [300]:
meta.bind

Engine(sqlite:///orm2.db)

In [302]:
# 이거를 해줘야 아래에서 오류가 안 뜸. SQLite를 조회하고 있던 프로그램을 끄는 것.
con.close()
#con = engine.connect()

2023-03-08 00:39:25,758 INFO sqlalchemy.engine.Engine ROLLBACK


In [304]:
con = engine.connect()

In [308]:
meta.create_all(engine)

2023-03-08 00:41:01,974 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:41:01,975 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2023-03-08 00:41:01,976 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:41:01,976 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ADDRESS")
2023-03-08 00:41:01,977 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:41:01,978 INFO sqlalchemy.engine.Engine COMMIT


In [309]:
Addr = meta.tables['T_ADDRESS']

In [310]:
type(User), type(Addr)

(sqlalchemy.sql.schema.Table, sqlalchemy.sql.schema.Table)

In [311]:
print(select(User, Addr))
# cross join 형태

SELECT "T_USER"."PK", "T_USER"."NAME", "T_ADDRESS"."PK" AS "PK_1", "T_ADDRESS"."ADDR", "T_ADDRESS"."FK" 
FROM "T_USER", "T_ADDRESS"


In [312]:
from sqlalchemy.sql import join

In [313]:
print(join(User, Addr, User.c.PK == Addr.c.FK))

"T_USER" JOIN "T_ADDRESS" ON "T_USER"."PK" = "T_ADDRESS"."FK"


In [314]:
print(select(User).join(Addr, User.c.PK == Addr.c.FK))
# 위 구문과 합쳐야 함

SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER" JOIN "T_ADDRESS" ON "T_USER"."PK" = "T_ADDRESS"."FK"


In [315]:
print(select(User.c.NAME, Addr.c.ADDR))

SELECT "T_USER"."NAME", "T_ADDRESS"."ADDR" 
FROM "T_USER", "T_ADDRESS"


In [316]:
print(select(User.c.NAME, Addr.c.ADDR)\
    .select_from(join(User, Addr, User.c.PK==Addr.c.FK)))
# ON 절이 자동적으로 생성됨

SELECT "T_USER"."NAME", "T_ADDRESS"."ADDR" 
FROM "T_USER" JOIN "T_ADDRESS" ON "T_USER"."PK" = "T_ADDRESS"."FK"


In [255]:
# 위에서 끊었기 떄문에 다시 엔진에 연결
con = engine.connect()

In [317]:
sql = select(User.c.NAME, Addr.c.ADDR)\
    .select_from(join(User, Addr, User.c.PK==Addr.c.FK))
cur = con.execute(sql)
cur.fetchall()
# 로그를 보고 정상적으로 수행이 됐음을 확인하면 됨.

2023-03-08 00:41:36,781 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:41:36,782 INFO sqlalchemy.engine.Engine SELECT "T_USER"."NAME", "T_ADDRESS"."ADDR" 
FROM "T_USER" JOIN "T_ADDRESS" ON "T_USER"."PK" = "T_ADDRESS"."FK"
2023-03-08 00:41:36,782 INFO sqlalchemy.engine.Engine [generated in 0.00173s] ()


[]

In [318]:
print(select(User.c.NAME, Addr.c.ADDR)\
    .select_from(join(User, Addr, User.c.PK==Addr.c.FK))\
    .where(User.c.PK==1))

SELECT "T_USER"."NAME", "T_ADDRESS"."ADDR" 
FROM "T_USER" JOIN "T_ADDRESS" ON "T_USER"."PK" = "T_ADDRESS"."FK" 
WHERE "T_USER"."PK" = :PK_1


In [226]:
print(insert(Addr))

INSERT INTO "T_ADDRESS" ("PK", "ADDR", "FK") VALUES (:PK, :ADDR, :FK)


In [319]:
print(insert(Addr).values(ADDR='아무거나', FK=1).compile().params)

{'ADDR': '아무거나', 'FK': 1}


In [320]:
print(User.select().where(User.c.PK==1))
print('')
print(select(User))

SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER" 
WHERE "T_USER"."PK" = :PK_1

SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER"


In [321]:
con.execute(insert(Addr).values(ADDR='아무거나', FK=1))

2023-03-08 00:42:50,555 INFO sqlalchemy.engine.Engine INSERT INTO "T_ADDRESS" ("ADDR", "FK") VALUES (?, ?)
2023-03-08 00:42:50,556 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ('아무거나', 1)


In [325]:
con.execute(insert(User).values(NAME='아무나'))

2023-03-08 00:43:54,591 INFO sqlalchemy.engine.Engine INSERT INTO "T_USER" ("NAME") VALUES (?)
2023-03-08 00:43:54,592 INFO sqlalchemy.engine.Engine [cached since 328.2s ago] ('아무나',)


In [330]:
meta.create_all(engine)

2023-03-08 00:44:20,767 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:44:20,768 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2023-03-08 00:44:20,768 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:44:20,769 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ADDRESS")
2023-03-08 00:44:20,769 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:44:20,770 INFO sqlalchemy.engine.Engine COMMIT


In [327]:
print(con.execute(select(User)).fetchall())
# 아무것도 안 들어가 있음

2023-03-08 00:43:59,011 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER"
2023-03-08 00:43:59,012 INFO sqlalchemy.engine.Engine [cached since 323.9s ago] ()
[(1, '아무나')]


In [328]:
FK = con.execute(User.select().where(User.c.PK == 1)).fetchone()[0]
# 1. User객체 - SELECT - WHERE => SQL문
# 2. engine의 connection pool onnection -> con -> con.execute 실행
# 3. 2번의 결과가 resultproxy -> cursor
# 4. 3의 결과를(커서가 가리키는 곳에 DB 결과가 있음, DB의 실제 데이터) fetchone(1행의 1열 가져온 것 = PK 값)

2023-03-08 00:44:03,987 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK", "T_USER"."NAME" 
FROM "T_USER" 
WHERE "T_USER"."PK" = ?
2023-03-08 00:44:03,992 INFO sqlalchemy.engine.Engine [generated in 0.00449s] (1,)


In [329]:
cur = con.execute(insert(Addr).values(ADDR='아무거나', FK=FK))
cur.lastrowid

2023-03-08 00:44:08,345 INFO sqlalchemy.engine.Engine INSERT INTO "T_ADDRESS" ("ADDR", "FK") VALUES (?, ?)
2023-03-08 00:44:08,346 INFO sqlalchemy.engine.Engine [cached since 77.79s ago] ('아무거나', 1)


2

In [331]:
con.commit()
# 이게 있어야 실제 DB에 반영이 됨

2023-03-08 00:44:44,212 INFO sqlalchemy.engine.Engine COMMIT


In [332]:
con.close()
meta.clear()
engine.dispose()

In [333]:
engine = create_engine('sqlite:///orm2.db', echo=True) # 기존에 작업해놓은 DB
meta = MetaData() # bind
con = engine.connect()

In [334]:
meta.create_all(engine)

2023-03-08 00:45:09,722 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:45:09,723 INFO sqlalchemy.engine.Engine COMMIT


In [143]:
# con.close()
# con = engine.connect()

In [335]:
meta.tables

FacadeDict({})

In [336]:
Table('T_USER', meta,
      Column('PK', Integer))

Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>), schema=None)

In [337]:
meta.create_all(engine)
# 나는 engine 줘야 함

2023-03-08 00:45:16,690 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:45:16,691 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2023-03-08 00:45:16,691 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:45:16,692 INFO sqlalchemy.engine.Engine COMMIT


In [338]:
con.execute(meta.tables['T_USER'].select()).fetchall()

2023-03-08 00:45:18,588 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:45:18,591 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" 
FROM "T_USER"
2023-03-08 00:45:18,592 INFO sqlalchemy.engine.Engine [generated in 0.00356s] ()


[(1,)]

In [339]:
User

Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>, primary_key=True, nullable=False), Column('NAME', String(), table=<T_USER>, nullable=False), schema=None)

In [346]:
# 원래는 안되는게 맞는 것
Table('T_USER', meta,
      Column('PK', Integer),
      Column('TEST', String),
      extend_existing=True)

# 메모리에서 관리하는 db 를 수정할 수 있다는 것

Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>), Column('TEST', String(), table=<T_USER>), schema=None)

In [347]:
meta.bind = engine

In [348]:
meta.create_all(engine)
# 나는 engine 줘야 함

2023-03-08 00:46:35,642 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:46:35,643 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2023-03-08 00:46:35,643 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:46:35,645 INFO sqlalchemy.engine.Engine COMMIT


In [349]:
con.commit()

2023-03-08 00:46:44,808 INFO sqlalchemy.engine.Engine COMMIT


In [350]:
meta.bind

Engine(sqlite:///orm2.db)

In [351]:
meta.tables

FacadeDict({'T_USER': Table('T_USER', MetaData(), Column('PK', Integer(), table=<T_USER>), Column('TEST', String(), table=<T_USER>), schema=None)})

In [352]:
con.execute(select(meta.tables['T_USER'])).fetchall()

2023-03-08 00:46:56,227 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:46:56,231 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" 
FROM "T_USER"
2023-03-08 00:46:56,234 INFO sqlalchemy.engine.Engine [cached since 97.65s ago] ()


[(1,)]

In [87]:
meta.create_all(engine)

2023-03-07 13:58:27,502 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-07 13:58:27,503 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2023-03-07 13:58:27,503 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 13:58:27,504 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
# 이런 과정들이 헷갈리는 이유
물리적 DB <----------> in memory Class-Instance 가 달라서.
            ORM - Core
                            MetaData
engine-dialect
T_USER                      T_USER(meta) (없는 Column)

#### 예제 시작

In [353]:
con.close()
meta.clear()
engine.dispose()

2023-03-08 00:48:16,222 INFO sqlalchemy.engine.Engine ROLLBACK


In [354]:
!ls

03.02(오후).ipynb           orm2.db
03.03(오전).ipynb           playlist.db
03.03(오후).ipynb           playlist2.db
03.06(오전).ipynb           playlist4.db
03.06(오후).ipynb           project_exercise.py
03.07(오전).ipynb           sns.db
03.07(오후).ipynb           sns2.db
example1.db                 test.db
example2.db                 test1.db
example3.db                 강의자료
orm1.db


In [355]:
engine = create_engine('sqlite:///example3.db', echo=True) # 기존에 작업해놓은 DB
meta = MetaData() # bind
con = engine.connect()

In [356]:
# 구조
# city, part, sells, supplier
# pk, name      fk1,fk2,price       fk
Table('city', meta,
      Column('pk', Integer, primary_key=True),
      Column('name', String), extend_existing=True)
Table('part', meta,
      Column('pk', Integer, primary_key=True),
      Column('name', String), extend_existing=True)
Table('supplier', meta,
      Column('pk', Integer, primary_key=True),
      Column('name', String),
      Column('fk', Integer), extend_existing=True)
Table('sells', meta,
      Column('fk1', Integer),
      Column('fk2', Integer),
      Column('price', Integer), extend_existing=True)

Table('sells', MetaData(), Column('fk1', Integer(), table=<sells>), Column('fk2', Integer(), table=<sells>), Column('price', Integer(), table=<sells>), schema=None)

In [357]:
meta.create_all(engine)

2023-03-08 00:48:57,459 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:48:57,460 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("city")
2023-03-08 00:48:57,460 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:48:57,462 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("part")
2023-03-08 00:48:57,463 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:48:57,464 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("supplier")
2023-03-08 00:48:57,464 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:48:57,465 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sells")
2023-03-08 00:48:57,465 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:48:57,466 INFO sqlalchemy.engine.Engine COMMIT


In [358]:
con.execute(select(meta.tables['sells'])).fetchall() # sells 에 아무것도 없음

2023-03-08 00:49:01,038 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:49:01,039 INFO sqlalchemy.engine.Engine SELECT sells.fk1, sells.fk2, sells.price 
FROM sells
2023-03-08 00:49:01,039 INFO sqlalchemy.engine.Engine [generated in 0.00176s] ()


[(1, 1, 4500),
 (1, 2, 4500),
 (1, 2, 5000),
 (2, 1, 4700),
 (2, 2, 4900),
 (3, 4, 5300),
 (3, 5, 2300),
 (3, 1, 900)]

In [359]:
city = meta.tables['city']
supplier = meta.tables['supplier']
part = meta.tables['part']
sells = meta.tables['sells']

In [360]:
sql = select(city.c.name, supplier.c.name)\
    .select_from(join(city, supplier, city.c.pk == supplier.c.fk))
con.execute(sql).fetchall()

2023-03-08 00:49:23,131 INFO sqlalchemy.engine.Engine SELECT city.name, supplier.name AS name_1 
FROM city JOIN supplier ON city.pk = supplier.fk
2023-03-08 00:49:23,132 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ()


[('성북구', '안암 1호점'), ('성북구', '안암 2호점'), ('성북구', '종암 1호점')]

In [ ]:
# supplier - sells - part 3개 조인
# city - supplier - sells - part 4개 조인
# select 할 columns 지정도 해보고
# where 조건도 표현해보고
# groupby, orderby => 할 수 있음 해보고(PPT에 있음)

#! 여기까지 코어를 실행한 것

In [361]:
# 플레이리스트 예제
engine = create_engine('sqlite:///playlist4.db', echo=True)
meta = MetaData()
con = engine.connect()

In [362]:
# 기존에 있던 DB 불러오는 것
meta.reflect(engine)

2023-03-08 00:50:20,160 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:50:20,161 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-03-08 00:50:20,161 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:50:20,163 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-03-08 00:50:20,164 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:50:20,164 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("album")
2023-03-08 00:50:20,165 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:50:20,166 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("artist")
2023-03-08 00:50:20,166 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 00:50:20,167 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("genre")
2023-03-08 00:50:20,167 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-0

In [363]:
meta.tables.keys()

dict_keys(['album', 'artist', 'genre', 'track'])

In [364]:
album = meta.tables['album']
artist = meta.tables['artist']

In [365]:
sql = select(album.c.name, artist.c.name)\
    .select_from(join(album, artist, album.c.fk==artist.c.pk))
con.execute(sql).fetchall()

2023-03-08 00:50:53,795 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 00:50:53,796 INFO sqlalchemy.engine.Engine SELECT album.name, artist.name AS name_1 
FROM album JOIN artist ON album.fk = artist.pk
2023-03-08 00:50:53,796 INFO sqlalchemy.engine.Engine [generated in 0.00135s] ()


[('앨범1', '가수1'),
 ('앨범2', '가수2'),
 ('앨범3', '가수3'),
 ('앨범4', '가수4'),
 ('싱글1', '가수1'),
 ('싱글2', '가수2'),
 ('싱글3', '가수3'),
 ('싱글4', '가수4')]

In [366]:
album.columns.keys(), artist.columns.keys()
# 전 수업에서 commit 안해서 데이터가 없음

(['pk', 'name', 'fk'], ['pk', 'name'])

In [367]:
con.execute(album.select()).fetchall()

2023-03-08 00:51:05,978 INFO sqlalchemy.engine.Engine SELECT album.pk, album.name, album.fk 
FROM album
2023-03-08 00:51:05,979 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ()


[(1, '앨범1', 1),
 (2, '앨범2', 2),
 (3, '앨범3', 3),
 (4, '앨범4', 4),
 (5, '싱글1', 1),
 (6, '싱글2', 2),
 (7, '싱글3', 3),
 (8, '싱글4', 4)]

In [370]:
# 데이터 넣어주기!!
con.execute(insert(album).values([{'name':'아무개1', 'fk':1}, {'name':'아무개2', 'fk':2}]))
#con.execute(insert(album).values({'name':'아무개2', 'fk':2}))

2023-03-08 00:54:05,420 INFO sqlalchemy.engine.Engine INSERT INTO album (name, fk) VALUES (?, ?), (?, ?)
2023-03-08 00:54:05,420 INFO sqlalchemy.engine.Engine [no key 0.00083s] ('아무개1', 1, '아무개2', 2)


In [371]:
# 데이터 넣어주기!!
con.execute(insert(artist).values([{'name':'가수1'},{'name':'가수2'}]))
#con.execute(insert(artist).values({'name':'가수2'}))

2023-03-08 00:54:45,020 INFO sqlalchemy.engine.Engine INSERT INTO artist (name) VALUES (?), (?)
2023-03-08 00:54:45,020 INFO sqlalchemy.engine.Engine [no key 0.00084s] ('가수1', '가수2')


In [372]:
sql = select(album.c.name, artist.c.name)\
    .select_from(join(album, artist, album.c.fk==artist.c.pk))
con.execute(sql).fetchall()

2023-03-08 00:54:51,772 INFO sqlalchemy.engine.Engine SELECT album.name, artist.name AS name_1 
FROM album JOIN artist ON album.fk = artist.pk
2023-03-08 00:54:51,772 INFO sqlalchemy.engine.Engine [cached since 238s ago] ()


[('앨범1', '가수1'),
 ('앨범2', '가수2'),
 ('앨범3', '가수3'),
 ('앨범4', '가수4'),
 ('싱글1', '가수1'),
 ('싱글2', '가수2'),
 ('싱글3', '가수3'),
 ('싱글4', '가수4'),
 ('아무개1', '가수1'),
 ('아무개2', '가수2')]

In [105]:
con.close()
meta.clear()
engine.dispose()

2023-03-07 14:03:21,574 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
# 기본적인 CORE 에서의 사용법을 배웠음 (지금까지)

In [ ]:
# 항상 DB 만들면 COMMIT 해주기!!

In [373]:
print(insert(artist))
# 이것은 실제 적용되는 것이 아니라, 전달되는 SQL 문이라는 것을 알아야 함.

INSERT INTO artist (pk, name) VALUES (:pk, :name)


In [374]:
print(insert(artist).values(name='hjahaha')),\
print(insert(artist).values(name='hjahaha').compile().params)

INSERT INTO artist (name) VALUES (:name)
{'name': 'hjahaha'}


(None, None)

In [ ]:
# 파라미터 수에 따라 컬럼이 결정됨.

In [375]:
con.execute(artist.update().values(name='가수1').where(artist.c.pk==1))

2023-03-08 00:55:25,745 INFO sqlalchemy.engine.Engine UPDATE artist SET name=? WHERE artist.pk = ?
2023-03-08 00:55:25,746 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ('가수1', 1)


In [376]:
print(con.execute(artist.update().values(name='가수1').where(artist.c.pk==1)))

2023-03-08 00:55:30,096 INFO sqlalchemy.engine.Engine UPDATE artist SET name=? WHERE artist.pk = ?
2023-03-08 00:55:30,097 INFO sqlalchemy.engine.Engine [cached since 4.352s ago] ('가수1', 1)


In [ ]:
# 실제 커넥션 풀에서 왔다갔다 해야 DB에 적용되는 것을 기억

In [119]:
con.execute(artist.select()).fetchall()

2023-03-07 15:05:12,469 INFO sqlalchemy.engine.Engine SELECT artist.pk, artist.name 
FROM artist
2023-03-07 15:05:12,473 INFO sqlalchemy.engine.Engine [generated in 0.00380s] ()


[(1, '가수1'), (2, '가수2'), (3, '가수3'), (4, '가수4')]

In [ ]:
sqlalchemy.__version__

In [120]:
print(artist.join(album, artist.c.pk==album.c.fk))

artist JOIN album ON artist.pk = album.fk


In [ ]:
# 131p 그림에 주목할 것.
# 이제부터는 위의 단을 만질 것. 
# 매핑

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
base = declarative_base()

In [ ]:
base.metadata.tables

In [ ]:
# 이제부터 코어단이 알아서 해주기 때문에 우리는 완전 편해진다.
# 인스턴스만 생성해주면 됨.
# Session 이 감시 -> 값의 변화가 있을 때.

In [ ]:
# 엔진이 커넥션 풀이랑 dialect를 들고 있음.